In [2]:
!pip install ultralytics

In [3]:
import os
print(os.listdir("/kaggle/input"))  # watch for 'radroad-anomaly-detection'
print(os.listdir("/kaggle/input/radroad-anomaly-detection"))
print(os.listdir("/kaggle/input/radroad-anomaly-detection/images"))
print(os.listdir("/kaggle/input/radroad-anomaly-detection/images/test"))  # should show ['images', 'labels']


['radroad-anomaly-detection']
['images', 'videos_without_audio']
['data.yaml', 'valid', 'test', 'train']
['labels', 'images']


In [4]:
import os
from ultralytics import YOLO

# Adjust only if Step 1 shows different names
ORIG_IMAGES_DIR = "/kaggle/input/radroad-anomaly-detection/images/test/images"
ORIG_LABELS_DIR = "/kaggle/input/radroad-anomaly-detection/images/test/labels"

# Output/working dirs
WORK_IMG_DIR = "/kaggle/working/images/test"
WORK_LBL_DIR = "/kaggle/working/labels/test"
FILTER_TMP_DIR = "/kaggle/working/labels/test_damages_only"

ORIGINAL_CLASS_ID = 2  # 'Road Damages' in original dataset → will remap to 0

# Guards
for p in [ORIG_IMAGES_DIR, ORIG_LABELS_DIR]:
    if not os.path.isdir(p):
        raise FileNotFoundError(f"Missing: {p}")

os.makedirs(FILTER_TMP_DIR, exist_ok=True)
os.makedirs(WORK_IMG_DIR, exist_ok=True)
os.makedirs(WORK_LBL_DIR, exist_ok=True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
def filter_labels(original_dir, new_dir, orig_cls=ORIGINAL_CLASS_ID):
    for filename in os.listdir(original_dir):
        src = os.path.join(original_dir, filename)
        if not os.path.isfile(src):
            continue
        new_lines = []
        with open(src, "r") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                if line.startswith(str(orig_cls) + " "):
                    new_lines.append("0" + line[len(str(orig_cls)):])
        if new_lines:
            with open(os.path.join(new_dir, filename), "w") as out:
                out.writelines(l + "\n" if not l.endswith("\n") else l for l in new_lines)

print("Filtering labels...")
filter_labels(ORIG_LABELS_DIR, FILTER_TMP_DIR)
print("Done filtering.")


Filtering labels...
Done filtering.


In [6]:
# Link/copy images to working images folder
for fname in os.listdir(ORIG_IMAGES_DIR):
    s = os.path.join(ORIG_IMAGES_DIR, fname)
    d = os.path.join(WORK_IMG_DIR, fname)
    if os.path.isfile(s) and not os.path.exists(d):
        try:
            os.symlink(s, d)  # fast; same-session only
        except OSError:
            import shutil
            shutil.copy2(s, d)

# Move filtered labels into working labels folder
for fname in os.listdir(FILTER_TMP_DIR):
    s = os.path.join(FILTER_TMP_DIR, fname)
    d = os.path.join(WORK_LBL_DIR, fname)
    if os.path.isfile(s):
        os.replace(s, d)

# Quick sanity: count paired samples
img_basenames = {os.path.splitext(f)[0] for f in os.listdir(WORK_IMG_DIR) if os.path.isfile(os.path.join(WORK_IMG_DIR, f))}
lbl_basenames = {os.path.splitext(f)[0] for f in os.listdir(WORK_LBL_DIR) if os.path.isfile(os.path.join(WORK_LBL_DIR, f))}
paired = len(img_basenames & lbl_basenames)
print("Images:", len(img_basenames), "Labels:", len(lbl_basenames), "Paired:", paired)


Images: 1263 Labels: 317 Paired: 317


In [7]:
%%writefile road_damages_only.yaml
train: /kaggle/working/images/test
val: /kaggle/working/images/test

nc: 1
names:
  0: Road Damages


Writing road_damages_only.yaml


In [8]:
model = YOLO("yolov8l.pt")
print("Starting training...")
results = model.train(
    data="road_damages_only.yaml",
    epochs=50,
    imgsz=640,
    project="road_damage_detection",
    name="run_damages_only"
)
print("Training finished!")


Starting training...
Ultralytics 8.3.204 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=road_damages_only.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_damages_only, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

/usr/local/lib/python3.11/dist-packages/ultralytics/engine/validator.py:288: RuntimeWarning: invalid value encountered in greater_equal
  matches = np.nonzero(iou >= threshold)  # IoU > threshold and classes match


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 2.5it/s 16.3s0.4s
                   all       1263        561    0.00158      0.517    0.00111   0.000493

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/50        10G      2.358      2.696      1.634          9        640: 100% ━━━━━━━━━━━━ 79/79 1.1it/s 1:090.9s3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 10% ━─────────── 4/40 1.9it/s 1.6s<18.9s

/usr/local/lib/python3.11/dist-packages/ultralytics/engine/validator.py:288: RuntimeWarning: invalid value encountered in greater_equal
  matches = np.nonzero(iou >= threshold)  # IoU > threshold and classes match


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 2.5it/s 15.8s0.4s
                   all       1263        561          0          0          0          0

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/50        10G      2.366      2.999      1.682          7        640: 100% ━━━━━━━━━━━━ 79/79 1.1it/s 1:100.9sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 2.4it/s 16.3s0.4s
                   all       1263        561      0.276      0.194       0.14     0.0556

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       4/50      9.98G      2.305      2.803      1.637          8        640: 100% ━━━━━━━━━━━━ 79/79 1.1it/s 1:100.9sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 40/40 2.4it/s 16.5s0.4s
                   all  

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       1263        561      0.887      0.838       0.91      0.533
Speed: 0.1ms preprocess, 10.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /kaggle/working/road_damage_detection/run_damages_only
Training finished!


In [ ]:
import os, shutil

wdir = "/kaggle/working/road_damage_detection/run_damages_only/weights"
print("Weights dir exists:", os.path.isdir(wdir))
print("Files:", os.listdir(wdir))

src = os.path.join(wdir, "best.pt")
dst = "/kaggle/working/best.pt"
if os.path.exists(src):
    shutil.copy2(src, dst)
    print("Copied to:", dst)
